# 당뇨병 데이터를 이용한 회귀분석 모델 비교

당뇨병과 관련된 요소들이 있고, 1년 후 값들을 예측하는 문제입니다.
- 알고리즘: KNN 이웃, 의사결정트리, 랜덤포레스트 등은 분류뿐만 아니라 회귀도 지원
- Ridge, Lasso 정규화 회귀도 사용


In [13]:
# 필요한 라이브러리 import
from sklearn.datasets import load_diabetes  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# xgboost는 별도 설치 필요: conda install xgboost
try:
    from xgboost import XGBRegressor
    XGBOOST_AVAILABLE = True
except ImportError:
    print("XGBoost가 설치되지 않았습니다. 'conda install xgboost'로 설치하세요.")
    XGBOOST_AVAILABLE = False


In [14]:
# 데이터 로드 및 탐색
data = load_diabetes()  # bunch 라는 클래스 타입으로 정리해서 준다
# 이상치, 누락치, 정규화까지 다 된 자료를 준다 - pandas, numpy

print("데이터 키들:", data.keys()) 
print("타겟 데이터 처음 10개:", data.target[:10])
print("특성 데이터 처음 10개:", data.data[:10])


데이터 키들: dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])
타겟 데이터 처음 10개: [151.  75. 141. 206. 135.  97. 138.  63. 110. 310.]
특성 데이터 처음 10개: [[ 0.03807591  0.05068012  0.06169621  0.02187239 -0.0442235  -0.03482076
  -0.04340085 -0.00259226  0.01990749 -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 -0.02632753 -0.00844872 -0.01916334
   0.07441156 -0.03949338 -0.06833155 -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 -0.00567042 -0.04559945 -0.03419447
  -0.03235593 -0.00259226  0.00286131 -0.02593034]
 [-0.08906294 -0.04464164 -0.01159501 -0.03665608  0.01219057  0.02499059
  -0.03603757  0.03430886  0.02268774 -0.00936191]
 [ 0.00538306 -0.04464164 -0.03638469  0.02187239  0.00393485  0.01559614
   0.00814208 -0.00259226 -0.03198764 -0.04664087]
 [-0.09269548 -0.04464164 -0.04069594 -0.01944183 -0.06899065 -0.07928784
   0.04127682 -0.0763945  -0.04117617 -0.09634616]
 [-0.04547248  0.05068012 -0.04716281 -0.015

In [15]:
print("\n데이터 설명:")
print(data.DESCR)


데이터 설명:
.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables

In [16]:
# 데이터 분리
X = data.data  # 현재 10개의 특성값
y = data.target  # 미래값으로 나타나는 것

print(f"X shape: {X.shape}")  # 442개이고 특성이 10개
print(f"y shape: {y.shape}")

# 데이터를 훈련/테스트 세트로 나눈다 (7.5:2.5 비율)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234)


X shape: (442, 10)
y shape: (442,)


## 1. Linear Regression (선형회귀)

선형회귀모델의 score 함수는 결정계수를 반환합니다:
- 1이면 완벽하게 예측
- 0이면 거의 예측불가
- 음수면 심각하게 안 맞는 상태


In [17]:
model = LinearRegression()
model.fit(X_train, y_train)  # 학습
y_pred = model.predict(X_test)

print("=== Linear Model ===")
print(f"훈련셋 R²: {model.score(X_train, y_train):.4f}")
print(f"테스트셋 R²: {model.score(X_test, y_test):.4f}")


=== Linear Model ===
훈련셋 R²: 0.5304
테스트셋 R²: 0.4694


## 2. Ridge Regression (릿지 회귀)

L2 정규화를 사용한 선형회귀


In [18]:
model = Ridge(alpha=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== Ridge Model ===")
print(f"훈련셋 R²: {model.score(X_train, y_train):.4f}")
print(f"테스트셋 R²: {model.score(X_test, y_test):.4f}")


=== Ridge Model ===
훈련셋 R²: 0.5225
테스트셋 R²: 0.4723


## 3. Lasso Regression (라쏘 회귀)

L1 정규화를 사용한 선형회귀

In [19]:
model = Lasso(alpha=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== Lasso Model ===")
print(f"훈련셋 R²: {model.score(X_train, y_train):.4f}")
print(f"테스트셋 R²: {model.score(X_test, y_test):.4f}")


=== Lasso Model ===
훈련셋 R²: 0.5202
테스트셋 R²: 0.4752


## 4. Decision Tree Regressor (의사결정트리 회귀)

의사결정트리는 회귀 가능하지만, 트리 계열은 언제나 과대적합 상태임


In [20]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== DecisionTreeRegressor Model ===")
print(f"훈련셋 R²: {model.score(X_train, y_train):.4f}")
print(f"테스트셋 R²: {model.score(X_test, y_test):.4f}")  # 회귀분석에서 score가 음수면 위험
print("특성의 중요도:", model.feature_importances_)


=== DecisionTreeRegressor Model ===
훈련셋 R²: 1.0000
테스트셋 R²: -0.0732
특성의 중요도: [0.0547852  0.01930743 0.36886387 0.08001894 0.04362728 0.07861217
 0.05050573 0.0179285  0.18137448 0.1049764 ]


## 5. Random Forest Regressor (랜덤포레스트 회귀)

의사결정트리의 업그레이드 버전, 여러 개의 분석기를 함께 사용하는 앙상블 방법
- 트리를 랜덤하게 많이 만들어서 평균값을 구함
- 할 때마다 별도의 트리가 만들어져서 계속 측정치가 달라짐
- `n_estimators`: 만들 트리 최대 개수
- `max_depth`: 트리의 최대 깊이 지정


In [21]:
model = RandomForestRegressor(random_state=0, n_estimators=300, max_depth=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== RandomForestRegressor Model ===")
print(f"훈련셋 R²: {model.score(X_train, y_train):.4f}")
print(f"테스트셋 R²: {model.score(X_test, y_test):.4f}")
print("특성의 중요도:", model.feature_importances_)


=== RandomForestRegressor Model ===
훈련셋 R²: 0.5810
테스트셋 R²: 0.4800
특성의 중요도: [0.0121268  0.00086328 0.47291228 0.06232497 0.00705143 0.01297284
 0.03647155 0.02186937 0.30277389 0.0706336 ]


## 6. Gradient Boosting Regressor (그라디언트 부스팅 회귀)

앙상블 계열, 약한 학습기들을 통해서 학습하고 보정작업을 거쳐서 결과를 찾아냄
- sklearn GradientBoostion, xgboost 라이브러리, LightGBM 등이 있음
- `learning_rate=0.1`: 학습률, 머신러닝이 학습하는 속도를 조절
  - 너무 높으면: 빨리 학습하다가 최적의 위치를 지나칠 수 있음
  - 너무 낮으면: 천천히 느리게 학습해서 최저점을 못 도달할 수 있음
- GridSearch: 하이퍼파라미터들을 주면 알아서 테스트하면서 적절한 파라미터를 찾아냄 (오래 걸림)


In [22]:
model = GradientBoostingRegressor(random_state=0, n_estimators=10, max_depth=3, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("=== GradientBoostingRegressor Model ===")
print(f"훈련셋 R²: {model.score(X_train, y_train):.4f}")
print(f"테스트셋 R²: {model.score(X_test, y_test):.4f}")
print("특성의 중요도:", model.feature_importances_)


=== GradientBoostingRegressor Model ===
훈련셋 R²: 0.5086
테스트셋 R²: 0.4397
특성의 중요도: [0.00994865 0.         0.49742687 0.04256156 0.01226501 0.0187062
 0.03427296 0.01095451 0.27842547 0.09543876]


## 7. XGBoost Regressor

설치 필요: `conda install xgboost`

Microsoft Visual C++ Build Tools 설치도 필요할 수 있음


In [23]:
if XGBOOST_AVAILABLE:
    model = XGBRegressor(random_state=0, n_estimators=10, max_depth=3, learning_rate=0.1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("=== XGBRegressor Model ===")
    print(f"훈련셋 R²: {model.score(X_train, y_train):.4f}")
    print(f"테스트셋 R²: {model.score(X_test, y_test):.4f}")
    print("특성의 중요도:", model.feature_importances_)
else:
    print("XGBoost가 설치되지 않아 건너뜁니다.")


=== XGBRegressor Model ===
훈련셋 R²: 0.4964
테스트셋 R²: 0.4179
특성의 중요도: [0.         0.06538191 0.3058761  0.05812455 0.02936009 0.05922715
 0.05762395 0.12139277 0.15938935 0.14362407]


## 모델 성능 비교 요약

위의 결과를 바탕으로 각 모델의 성능을 비교해보세요:
- 훈련셋 R²와 테스트셋 R²의 차이가 클수록 과적합
- 테스트셋 R²가 높을수록 일반화 성능이 좋음
- 특성 중요도를 통해 어떤 특성이 예측에 중요한지 확인 가능
